# Notebook 7: Agent con Integrazione Help Desk

**Obiettivo**: Implementare un Agent che integra con un sistema Help Desk tramite API, creando e gestendo ticket

---


## 1. Setup API Mock Help Desk

Creiamo un'API mock semplice con Flask per simulare un sistema Help Desk reale.


In [ ]:
# API Mock Help Desk (Flask)
from flask import Flask, jsonify, request
import threading
import time
from datetime import datetime

# Database in-memory per ticket (in produzione sarebbe un database reale)
tickets_db = {}
ticket_counter = 1

app = Flask(__name__)

@app.route('/api/tickets', methods=['POST'])
def create_ticket():
    """Crea un nuovo ticket"""
    global ticket_counter
    
    data = request.json
    ticket_id = f"TICKET-{ticket_counter:04d}"
    ticket_counter += 1
    
    ticket = {
        "id": ticket_id,
        "title": data.get("title", "Nessun titolo"),
        "description": data.get("description", ""),
        "status": "open",
        "priority": data.get("priority", "medium"),
        "created_at": datetime.now().isoformat(),
        "updated_at": datetime.now().isoformat()
    }
    
    tickets_db[ticket_id] = ticket
    return jsonify(ticket), 201

@app.route('/api/tickets/<ticket_id>', methods=['GET'])
def get_ticket(ticket_id):
    """Recupera un ticket per ID"""
    if ticket_id in tickets_db:
        return jsonify(tickets_db[ticket_id]), 200
    else:
        return jsonify({"error": "Ticket non trovato"}), 404

@app.route('/api/tickets/<ticket_id>', methods=['PUT'])
def update_ticket(ticket_id):
    """Aggiorna un ticket"""
    if ticket_id not in tickets_db:
        return jsonify({"error": "Ticket non trovato"}), 404
    
    data = request.json
    ticket = tickets_db[ticket_id]
    
    # Aggiorna campi se forniti
    if "status" in data:
        ticket["status"] = data["status"]
    if "priority" in data:
        ticket["priority"] = data["priority"]
    if "description" in data:
        ticket["description"] = data["description"]
    
    ticket["updated_at"] = datetime.now().isoformat()
    
    return jsonify(ticket), 200

@app.route('/api/tickets', methods=['GET'])
def search_tickets():
    """Cerca ticket (filtri opzionali)"""
    status = request.args.get("status")
    priority = request.args.get("priority")
    
    results = []
    for ticket in tickets_db.values():
        if status and ticket["status"] != status:
            continue
        if priority and ticket["priority"] != priority:
            continue
        results.append(ticket)
    
    return jsonify({"tickets": results, "count": len(results)}), 200

# Avvia server Flask in thread separato
def run_flask():
    app.run(host='127.0.0.1', port=5000, debug=False, use_reloader=False)

flask_thread = threading.Thread(target=run_flask, daemon=True)
flask_thread.start()

# Aspetta che server sia pronto
time.sleep(2)

print("✅ API Mock Help Desk avviata!")
print("  - Endpoint: http://127.0.0.1:5000")
print("  - POST /api/tickets - Crea ticket")
print("  - GET /api/tickets/<id> - Recupera ticket")
print("  - PUT /api/tickets/<id> - Aggiorna ticket")
print("  - GET /api/tickets?status=X - Cerca ticket")


## 2. Test API Mock

Verifichiamo che l'API funzioni correttamente prima di integrarla con l'Agent.


In [ ]:
import requests

# Test creazione ticket
response = requests.post(
    "http://127.0.0.1:5000/api/tickets",
    json={
        "title": "Problema login",
        "description": "Non riesco ad accedere al sistema",
        "priority": "high"
    }
)

if response.status_code == 201:
    ticket = response.json()
    print(f"✅ Ticket creato: {ticket['id']}")
    print(f"   Titolo: {ticket['title']}")
    print(f"   Stato: {ticket['status']}")
    print(f"   Priorità: {ticket['priority']}")
    
    # Test recupero ticket
    ticket_id = ticket['id']
    get_response = requests.get(f"http://127.0.0.1:5000/api/tickets/{ticket_id}")
    if get_response.status_code == 200:
        print(f"\n✅ Ticket recuperato: {get_response.json()['title']}")
else:
    print(f"❌ Errore: {response.status_code}")


## 3. Setup LangChain e LLM

Importiamo LangChain e configuriamo l'LLM per l'Agent.


In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage

# Inizializza LLM
llm = ChatOllama(
    model="llama3.2:3b",
    temperature=0.7
)

print("✅ LangChain e LLM configurati!")


## 4. Creare Tools Help Desk

Creiamo tools che chiamano l'API Help Desk per creare, recuperare e aggiornare ticket.


In [ ]:
# Tool 1: Crea ticket
@tool
def create_ticket(title: str, description: str, priority: str = "medium") -> str:
    """Crea un nuovo ticket nel sistema Help Desk.
    
    Args:
        title: Titolo del ticket (breve descrizione problema)
        description: Descrizione dettagliata del problema
        priority: Priorità ticket ("low", "medium", "high", "urgent")
    
    Returns:
        ID del ticket creato e informazioni base
    """
    try:
        response = requests.post(
            "http://127.0.0.1:5000/api/tickets",
            json={
                "title": title,
                "description": description,
                "priority": priority
            },
            timeout=5
        )
        
        if response.status_code == 201:
            ticket = response.json()
            return f"Ticket creato con successo! ID: {ticket['id']}, Titolo: {ticket['title']}, Stato: {ticket['status']}, Priorità: {ticket['priority']}"
        else:
            return f"Errore nella creazione ticket: {response.status_code} - {response.text}"
    except Exception as e:
        return f"Errore nella chiamata API: {str(e)}"

# Tool 2: Recupera ticket
@tool
def get_ticket(ticket_id: str) -> str:
    """Recupera informazioni su un ticket esistente.
    
    Args:
        ticket_id: ID del ticket (es. "TICKET-0001")
    
    Returns:
        Informazioni complete del ticket
    """
    try:
        response = requests.get(
            f"http://127.0.0.1:5000/api/tickets/{ticket_id}",
            timeout=5
        )
        
        if response.status_code == 200:
            ticket = response.json()
            return f"Ticket {ticket['id']}: {ticket['title']} | Stato: {ticket['status']} | Priorità: {ticket['priority']} | Creato: {ticket['created_at']}"
        else:
            return f"Ticket non trovato: {ticket_id}"
    except Exception as e:
        return f"Errore nella chiamata API: {str(e)}"

# Tool 3: Aggiorna ticket
@tool
def update_ticket(ticket_id: str, status: str = None, priority: str = None) -> str:
    """Aggiorna lo stato o la priorità di un ticket.
    
    Args:
        ticket_id: ID del ticket da aggiornare
        status: Nuovo stato ("open", "in_progress", "resolved", "closed")
        priority: Nuova priorità ("low", "medium", "high", "urgent")
    
    Returns:
        Conferma aggiornamento
    """
    try:
        update_data = {}
        if status:
            update_data["status"] = status
        if priority:
            update_data["priority"] = priority
        
        if not update_data:
            return "Errore: Fornire almeno status o priority da aggiornare"
        
        response = requests.put(
            f"http://127.0.0.1:5000/api/tickets/{ticket_id}",
            json=update_data,
            timeout=5
        )
        
        if response.status_code == 200:
            ticket = response.json()
            return f"Ticket {ticket_id} aggiornato: Stato={ticket['status']}, Priorità={ticket['priority']}"
        else:
            return f"Errore nell'aggiornamento: {response.status_code} - {response.text}"
    except Exception as e:
        return f"Errore nella chiamata API: {str(e)}"

# Tool 4: Cerca ticket
@tool
def search_tickets(status: str = None, priority: str = None) -> str:
    """Cerca ticket nel sistema Help Desk con filtri opzionali.
    
    Args:
        status: Filtra per stato ("open", "in_progress", "resolved", "closed")
        priority: Filtra per priorità ("low", "medium", "high", "urgent")
    
    Returns:
        Lista ticket trovati
    """
    try:
        params = {}
        if status:
            params["status"] = status
        if priority:
            params["priority"] = priority
        
        response = requests.get(
            "http://127.0.0.1:5000/api/tickets",
            params=params,
            timeout=5
        )
        
        if response.status_code == 200:
            data = response.json()
            tickets = data.get("tickets", [])
            count = data.get("count", 0)
            
            if count == 0:
                return "Nessun ticket trovato con i filtri specificati"
            
            result = f"Trovati {count} ticket:\n"
            for ticket in tickets:
                result += f"  - {ticket['id']}: {ticket['title']} ({ticket['status']}, {ticket['priority']})\n"
            
            return result
        else:
            return f"Errore nella ricerca: {response.status_code}"
    except Exception as e:
        return f"Errore nella chiamata API: {str(e)}"

print("✅ Tools Help Desk creati:")
print("  1. create_ticket - Crea nuovo ticket")
print("  2. get_ticket - Recupera ticket per ID")
print("  3. update_ticket - Aggiorna ticket")
print("  4. search_tickets - Cerca ticket con filtri")


In [ ]:
# Test creazione ticket
result1 = create_ticket.invoke({
    "title": "Test ticket",
    "description": "Ticket di test",
    "priority": "medium"
})
print(f"Test 1 - Creazione: {result1}\n")

# Test recupero ticket (usa ID dal test precedente)
# NOTA: In un notebook reale, estrai l'ID dal risultato precedente
test_ticket_id = "TICKET-0001"  # Sostituisci con ID reale se necessario
result2 = get_ticket.invoke({"ticket_id": test_ticket_id})
print(f"Test 2 - Recupero: {result2}\n")

# Test ricerca
result3 = search_tickets.invoke({"status": "open"})
print(f"Test 3 - Ricerca: {result3}\n")

print("✅ Tutti i tools Help Desk funzionano!")


## 6. Creare Agent Help Desk

Creiamo un Agent ReAct che può usare i tools Help Desk per gestire ticket.


In [ ]:
# Raccogli tutti i tools
tools = [create_ticket, get_ticket, update_ticket, search_tickets]

# Crea system prompt per Agent Help Desk
system_prompt = """Sei un assistente Help Desk che aiuta gli utenti a creare e gestire ticket.

Hai accesso ai seguenti tools:
- create_ticket: Crea un nuovo ticket nel sistema Help Desk
- get_ticket: Recupera informazioni su un ticket esistente
- update_ticket: Aggiorna lo stato o priorità di un ticket
- search_tickets: Cerca ticket con filtri (stato, priorità)

Quando un utente chiede di creare un ticket:
1. Usa create_ticket con titolo, descrizione e priorità appropriata
2. Conferma all'utente che il ticket è stato creato con l'ID

Quando un utente chiede informazioni su un ticket:
1. Usa get_ticket con l'ID fornito
2. Presenta le informazioni in modo chiaro

Quando un utente chiede di aggiornare un ticket:
1. Usa update_ticket con l'ID e i nuovi valori
2. Conferma l'aggiornamento

IMPORTANTE:
- Usa sempre i tools quando appropriato
- Fornisci ID ticket quando creati o aggiornati
- Se un tool fallisce, informa l'utente e suggerisci alternative"""

# Crea Agent con create_agent (LangChain 1.2+)
agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt=system_prompt,
    debug=True
)

print("✅ Agent Help Desk creato!")
print(f"Tools disponibili: {len(tools)}")


## 7. Test Agent Help Desk

Testiamo l'Agent con scenari reali di gestione ticket.


In [ ]:
# Test 1: Creazione ticket
query1 = "Voglio aprire un ticket per un problema di login. Il sistema non dice che la password che ho inserito è errata."
print(f"Query: {query1}\n")
result1 = agent.invoke({"messages": [HumanMessage(content=query1)]})
print(f"\nRisposta: {result1['messages'][-1].content}\n")
print("="*60)


In [ ]:
# Test 2: Query ticket (usa ID dal test precedente o crea nuovo)
# NOTA: In un notebook reale, estrai l'ID dal risultato precedente
query2 = "Dimmi lo stato del ticket TICKET-0003"
print(f"Query: {query2}\n")
result2 = agent.invoke({"messages": [HumanMessage(content=query2)]})
print(f"\nRisposta: {result2['messages'][-1].content}\n")
print("="*60)


In [ ]:
# Test 3: Aggiornamento ticket
query3 = "Aggiorna il ticket TICKET-0003 impostando lo stato a 'in_progress'"
print(f"Query: {query3}\n")
result3 = agent.invoke({"messages": [HumanMessage(content=query3)]})
print(f"\nRisposta: {result3['messages'][-1].content}\n")
print("="*60)


In [ ]:
# Test 4: Query complessa (crea + aggiorna)
query4 = "Crea un ticket urgente per un problema critico del sistema e poi dimmi l'ID"
print(f"Query: {query4}\n")
result4 = agent.invoke({"messages": [HumanMessage(content=query4)]})
print(f"\nRisposta: {result4['messages'][-1].content}\n")
print("="*60)


In [ ]:
# Test 5: Ricerca ticket
query5 = "Mostrami tutti i ticket aperti"
print(f"Query: {query5}\n")
result5 = agent.invoke({"messages": [HumanMessage(content=query5)]})
print(f"\nRisposta: {result5['messages'][-1].content}\n")
print("="*60)


## 8. Note e Best Practices

### Cosa abbiamo imparato:
1. **API Integration**: Come integrare Agent con API esterne (Help Desk)
2. **Tools API**: Creare tools che chiamano API REST
3. **Error Handling**: Gestire errori API nei tools
4. **Agent Orchestration**: Agent può combinare multiple operazioni
5. **API Mock**: Usare mock per test senza sistema reale

### Best Practices:

**API Integration**:
- Timeout: Usa timeout per evitare hang
- Error handling: Gestisci errori HTTP (404, 500, etc.)
- Retry logic: Considera retry per chiamate fallite
- Authentication

**Tools API**:
- Descrizioni chiare: Agent deve capire quando usare tool
- Validazione input: Valida parametri prima di chiamare API
- Messaggi utili: Restituisci messaggi chiari in caso di errore

**Testing**:
- Test API prima di integrare con Agent
- Test tools individualmente
- Test Agent con scenari reali

### Limitazioni e Considerazioni:
- **API Mock**: Questo notebook usa mock, in produzione serve API reale
- **Sicurezza**: Non hardcode API keys, usa variabili ambiente
- **Rate Limiting**: Rispetta rate limits API
- **Network**: Agent può essere lento se API è lenta

---

**Congratulazioni! Hai completato il Notebook 7! 🎉**